## Getting started with TensorFlow and machine learning
#### Brief words about machine learning and tensorflow
The need for machine learning arises when we have a lot of data and want to learn from it. The data is often given as input and output pairs and we want a computer to find a pattern (we assume there is one) between some input and given outputs. In simple cases, there can be a simple relationship between input and outputs such as the relationship between grams and kilograms which can be described as a linear function. But Machine learning gets more interesting and powerful when the relationship between inputs and outputs is not obvious at first. For example in this tutorial we will learn to map images of digits to their label. Pretty cool, right ?
According to the tensorflow website, this is considered the "hello world" of machine learning so let's get started. 

#### Methodology and approach to the problem
A machine learning problem can be broken down in four parts : acquiring the data, specifying the models (type of relationship between input and output should look for), training the computer to look for the best model among the set of models he has to explore and evaluating how well the model predicts output on new data.
##### The data
The first and essential part is the initial data. This data is what the computer will look at to teach himself how to find patterns between input and outputs. In most cases, the data is not given in a clean format and needs to be preprocessed so that the computer knows what to look for (features) and what to predict (outputs). A challenge in machine learning is to extract the features having the most predictive power in the data. <br />
This area is called feature engineering and is an important component of machine learning expertise. Recently, a form of machine learning called deep learning reduced the need for extensive human feature engineering allowing to learn complex outputs from raw data. In our cases, the data we will look at is the MNIST data which is made of black-and-white images of number (described by a 28*28 matrix of pixel values) and their labels (the number they represent). A first step is to download this data set in out repo. Thankfully, there is some already written code which does this job for us in the following next three lines.

In [1]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print "Hey"
import tensorflow as tf

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Hey


After downloading our data, we want to split it into input and output pairs. The simplest way to represent a 28*28 black-and-white image is as a list of 784 pixel values. For the output, each number is described as a one_hot vector (ith element is 1 if the label corresponds to element i). For example 3 will look like : [0,0,0,1,0,0,0,0,0,0].

In [2]:
image_vector = tf.placeholder("float", [None, 784]) 
label = tf.placeholder("float", [None, 10]) 

In the two previous lines of code, we told TensorFlow that one part of the data will come as a feature vector consisting of floating point number of 784 dimensions and another part of the data will come as a one-hot vector of 10 dimensions.

#### Modelling part 1 : Choosing the model
Now after having downloaded our data, we define the type of relationships between image_vectors and labels we want the computer to look for. In our case, the model is called a logistic regression. In logistic regression, we do a weighted sum of each feature in a feature vector and different labels correspond to different weights on each dimensions of the feature vector. This sum is indicative of how much the computer believes that one example belongs to a specific class. The output of this logistic regression model can be seen as a confidence score (more precisely a probability distribution) on each of the different type of label class. For example an output of [0.2,0,0.8,0,0,0,0,0,0,0] indicates a 80% confidence that the image is 2 and 20% confidence that the image is 0.


In [4]:
Weights = tf.Variable(tf.zeros([784, 10]))
bias = tf.Variable(tf.zeros([10]))
confidence_score = tf.nn.softmax(tf.matmul(image_vector, Weights) + bias) 
print "Model chosen"

Model chosen


#### Modelling part 2  : Choosing the loss function 
Now after having defined our model to find relationship between inputs and outputs, we want to find a way to measure how wrong or how right it is. In our case we use the cross-entropy loss function.

In [5]:
learning_rate = 0.02
loss_function = -tf.reduce_sum(label*tf.log(confidence_score)) 
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function) 
print "Loss function chosen"


Loss function chosen


After having defined our loss function, we can tell the program that the goal of the learning process is to learn the set of weights and biases which will minimize this loss function. 

#### Training : Showing labeled examples to the computer 
After defining all the variables, we initialize them to random values and part of the training process is to adjust these values as example arrive. To start showing examples, we now open a TensorFlow session. A TensorFlow session allows to run predefined tensorflow operations with a given set of resources. Since machine learning is compute-intensive we want to allocate resources just at the right moment, run all of our computations and deallocate these ressources.


In [7]:
init = tf.initialize_all_variables()
print "Done initializing"

Done initializing


Here we use mini-batch stochastic gradient descent where we use mini-batch Stochastic Gradient Descent (SGD) with a mini_batch_size of 30 and training_epoch of 1000.

In [ ]:
training_epochs = 1000
mini_batch_size = 30
display_step=1
print "Training parameters chosen"

Now we launch the computation graph

In [ ]:
# Launch the graph
print "Hey"
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        print epoch
        avg_loss = 0.
        total_batch_size = int(mnist.train.num_examples/mini_batch_size)
        # Loop over all batches
        for i in range(total_batch_size):
            batch_images, batch_labels = mnist.train.next_batch(mini_batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={image_vector: batch_images, label: batch_labels})
            # Compute average loss
            avg_loss += sess.run(loss_function, feed_dict={image_vector: batch_images, label: batch_labels})/total_batch_size
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "At epoch:", '%04d' % (epoch+1), "the average loss is", "{:.9f}".format(avg_loss)

    print "Training done"
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(labels, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({image_vector: mnist.test.images, labels: mnist.test.labels})